In [1]:
import codecs
import numpy as np
import tensorflow as tf
import pandas as pd 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AEW2rNDLc-sIylvc-fvGaMwO_fZEnB7Z6xVufVQct2sHQV5QBcisQc
Mounted at /content/drive


In [3]:
%cd "/content/drive/My Drive"

/content/drive/My Drive


In [4]:
dialogs = pd.read_csv('movie_conversations.txt', sep = '\+\+\+\$\+\+\+', header = None, engine='python',)
dialogs.head()

,0,1,2,3
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"
3,u0,u2,m0,"['L204', 'L205', 'L206']"
4,u0,u2,m0,"['L207', 'L208']"


In [5]:
dialogs[3] = dialogs[3].str.replace('[', '')
dialogs[3] = dialogs[3].str.replace(']', '')
dialogs[3] = dialogs[3].str.replace("'", '')
# dialogs[3] = dialogs[3].str.replace("L", '')
dialogs[3] = dialogs[3].str.strip()
# dialogs[3].apply()
dialogs.head()

,0,1,2,3
0,u0,u2,m0,"L194, L195, L196, L197"
1,u0,u2,m0,"L198, L199"
2,u0,u2,m0,"L200, L201, L202, L203"
3,u0,u2,m0,"L204, L205, L206"
4,u0,u2,m0,"L207, L208"


In [6]:
test = dialogs[3][301].split(sep = ', ')
test

['L3496', 'L3497']

In [7]:
replies = pd.read_csv('movie_lines.txt', sep = '\+\+\+\$\+\+\+', header = None, engine='python',index_col=0)#
replies.head()

,1,2,3,4
0,,,,
L1045,u0,m0,BIANCA,They do not!
L1044,u2,m0,CAMERON,They do to!
L985,u0,m0,BIANCA,I hope so.
L984,u2,m0,CAMERON,She okay?
L925,u0,m0,BIANCA,Let's go.


In [8]:
replies.index = replies.index.str.strip()
replies.index

Index(['L1045', 'L1044', 'L985', 'L984', 'L925', 'L924', 'L872', 'L871',
       'L870', 'L869',
       ...
       'L666262', 'L666522', 'L666521', 'L666520', 'L666372', 'L666371',
       'L666370', 'L666369', 'L666257', 'L666256'],
      dtype='object', name=0, length=304713)

In [9]:
train = []
labels = []
def make_lists(dialogs_str):
  replies_num = dialogs_str.split(sep = ', ')
  count = (len(replies_num))-1
  for i in range(count):
    train.append(str(replies.loc[[replies_num[i]]][4].values[0]).strip())
    labels.append(str(replies.loc[[replies_num[i+1]]][4].values[0]).strip())

# dialogs[3][:15000].apply(make_lists)
dialogs[3].apply(make_lists)
len(train)

221616

In [10]:
train[5:9]

['Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.']

In [11]:
labels[5:9]

["The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame."]

In [12]:
def prepare_vocab(texts):
    vocab = sorted(set(''.join(texts)))
    vocab.append('<START>')
    vocab.append('<END>')
    vocab_size = len(vocab)
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return vocab_size, char2idx, idx2char

INPUT_VOCAB_SIZE, input_char2idx, input_idx2char = prepare_vocab(train)
TARGET_VOCAB_SIZE, target_char2idx, target_idx2char = prepare_vocab(labels)

In [13]:
input_texts_as_int = [[input_char2idx[c] for c in text] for text in train]
target_texts_as_int = [[target_char2idx[c] for c in text] for text in labels]

encoder_input_seqs = [np.array(text) for text in input_texts_as_int]
decoder_input_seqs = []
decoder_target_seqs = []
for target_text in target_texts_as_int:
    decoder_input_seqs.append(np.array([target_char2idx['<START>']] + target_text))
    decoder_target_seqs.append(np.array(target_text + [target_char2idx['<END>']]))

In [14]:
encoder_input_seqs

[array([34, 63, 76,  1, 85, 67,  1, 75, 63, 73, 67,  1, 82, 70, 71, 81,  1,
        79, 83, 71, 65, 73, 31,  1,  1, 49, 77, 86, 63, 76, 76, 67,  1, 42,
        77, 80, 80, 71, 76, 67,  1, 63, 76, 66,  1, 32, 76, 66, 80, 67, 85,
         1, 33, 63, 80, 80, 67, 82, 82,  1, 63, 80, 67,  1, 70, 63, 84, 71,
        76, 69,  1, 63, 76,  1, 71, 76, 65, 80, 67, 66, 71, 64, 74, 87,  1,
        70, 77, 80, 80, 67, 76, 66, 77, 83, 81,  1, 78, 83, 64, 74, 71, 65,
         1, 64, 80, 67, 63, 73, 13,  1, 83, 78,  1, 77, 76,  1, 82, 70, 67,
         1, 79, 83, 63, 66, 14,  1,  1, 32, 69, 63, 71, 76, 14]),
 array([54, 67, 74, 74, 12,  1, 40,  1, 82, 70, 77, 83, 69, 70, 82,  1, 85,
        67,  8, 66,  1, 81, 82, 63, 80, 82,  1, 85, 71, 82, 70,  1, 78, 80,
        77, 76, 83, 76, 65, 71, 63, 82, 71, 77, 76, 12,  1, 71, 68,  1, 82,
        70, 63, 82,  8, 81,  1, 77, 73, 63, 87,  1, 85, 71, 82, 70,  1, 87,
        77, 83, 14]),
 array([45, 77, 82,  1, 82, 70, 67,  1, 70, 63, 65, 73, 71, 76, 69,  1, 63,


In [15]:
# max_enc_seq_length = max([len(seq) for seq in encoder_input_seqs])
# max_dec_seq_length = max([len(seq) for seq in decoder_input_seqs])

max_enc_seq_length = 253
max_dec_seq_length = 254


encoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    encoder_input_seqs,
    value=input_char2idx[' '],
    padding='post',
    maxlen=max_enc_seq_length)

decoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_input_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

decoder_target_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_target_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

In [16]:
decoder_target_seqs

array([[54, 67, 74, ...,  1,  1,  1],
       [45, 77, 82, ...,  1,  1,  1],
       [46, 73, 63, ...,  1,  1,  1],
       ...,
       [40,  1, 82, ...,  1,  1,  1],
       [32, 76, 66, ...,  1,  1,  1],
       [50, 77,  1, ...,  1,  1,  1]], dtype=int32)

In [17]:
H_SIZE = 251 # Размерность скрытого состояния LSTM
EMB_SIZE = 252 # размерность эмбеддингов (и для входных и для выходных цепочек)

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(INPUT_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.lstm1 = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)
        
    def call(self, x):
        out = self.embed(x)
        out, h, c = self.lstm(out)
        _, h1, c1 = self.lstm1(out)
        state = (h, c)
        state1 = (h1, c1)
        return (state, state1)

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(TARGET_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.lstm1 = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(TARGET_VOCAB_SIZE, activation='softmax')
        
    def call(self, x, init_state):
        out = self.embed(x)
        out, h, c = self.lstm(out, initial_state=init_state[0])
        out, h1, c1 = self.lstm1(out, initial_state=init_state[1])
        out = self.fc(out)
        state = (h, c)
        state1 = (h1, c1)
        return out, (state, state1)

encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
def seq2seq_inference(input_seq):
    state = encoder_model(input_seq)

    target_seq = np.array([[target_char2idx['<START>']]])

    decoded_sentence = ''
    while True:
        output_tokens, state = decoder_model(target_seq, state)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_idx2char[sampled_token_index]
        if (sampled_char == '<END>' or
          len(decoded_sentence) > max_dec_seq_length):
           break
        else:
          decoded_sentence += sampled_char


        target_seq = np.array([[sampled_token_index]])

    return decoded_sentence

In [19]:
def ask_my_bot(reply):
  test = [[input_char2idx[c] for c in reply]]
  test = [np.array(text) for text in test]
  test = tf.keras.preprocessing.sequence.pad_sequences(
      test,
      value=input_char2idx[' '],
      padding='post',
      maxlen=max_enc_seq_length)

  decoded_sentence = seq2seq_inference(test)
  print('-',reply)
  print('-',decoded_sentence)
ask_my_bot('Hello!')

- Hello!
- >>>D>D>D>h1<<xWW&&&&77))000vcccc11119999999999999999999999<START>9999999<START>99999<START>99999<START>99999<START>9999<START>99999<START>9999<START>9999<START>9999<START>9999<START>9999<START>9999<START>9999<START>9999<START>9999<START>9999<START>9999<START>


In [29]:
BATCH_SIZE = 128
EPOCHS = 5
loss = tf.losses.SparseCategoricalCrossentropy()
optimizer = tf.optimizers.RMSprop(learning_rate=0.0000001)
seq2seq.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# seq2seq.build()
# seq2seq.load_weights('chatbot_lstm_2_weights')

ask_my_bot('Hello!')
ask_my_bot('How are you?')
ask_my_bot('You know French?')
ask_my_bot('Bye')



- Hello!
- Hi.                                                                                                                                                                                                                                                            
- How are you?
- Fine.  I'm sorry.
- You know French?
- Sure.
- Bye
- You don't know what you're talking about.


In [30]:
# ask_my_bot('Hello!')
# ask_my_bot('How are you?')
# ask_my_bot('You know French?')
# ask_my_bot('Goodbye')

for i in range(20):

  seq2seq.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs,
            # steps_per_epoch=10,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS)
  
  ask_my_bot('Hello!')
  ask_my_bot('How are you?')
  ask_my_bot('You know French?')
  ask_my_bot('Goodbye')

  # tf.keras.models.save_model(seq2seq,'bot_weights')
  seq2seq.save_weights('chatbot_lstm_2_weights')

Epoch 1/5
1732/1732 [==============================] - 350s 202ms/step - loss: 0.2083 - accuracy: 0.9352
Epoch 2/5
1732/1732 [==============================] - 350s 202ms/step - loss: 0.2083 - accuracy: 0.9352
Epoch 3/5
1732/1732 [==============================] - 350s 202ms/step - loss: 0.2083 - accuracy: 0.9352
Epoch 4/5
1732/1732 [==============================] - 350s 202ms/step - loss: 0.2083 - accuracy: 0.9352
Epoch 5/5
1732/1732 [==============================] - 350s 202ms/step - loss: 0.2083 - accuracy: 0.9352
- Hello!
- Hi.                                                                                                                                                                                                                                                            
- How are you?
- Fine.  I'm sorry.
- You know French?
- Sure.
- Goodbye
- You don't know what you're talking about.
Epoch 1/5
1027/1732 [================>.............] - ETA: 2:22 - loss: 0.2090 - accuracy: 

KeyboardInterrupt: ignored

In [33]:
    test = 159
    input_seq = encoder_input_seqs[test:test+1]
    decoded_sentence = seq2seq_inference(input_seq)
    print('-')
    print('Input sentence:', train[test])
    print('Result sentence:', decoded_sentence)
    print('Target sentence:', labels[test])

-
Input sentence: What makes you think he'll do it?
Result sentence: I don't know.
Target sentence: He seems like he thrives on danger


In [31]:
ask_my_bot('You are welcome.')
ask_my_bot('I am talking about real life')
ask_my_bot('Oh, nevermind')
ask_my_bot('How old are you?')
ask_my_bot('I am not a cop.')
ask_my_bot('Go! Take Serge.. and watch your backs.')

ask_my_bot('How do I look?')
ask_my_bot('Okay, lets do it.')
ask_my_bot('What?')
ask_my_bot('On my way.')
ask_my_bot('Sure.  Go left.')
ask_my_bot('Good morning.')
ask_my_bot('I want to go back to Vienna.')
ask_my_bot('You ride like a Comanche.')
ask_my_bot('So we got a plan?')

- You are welcome.
- It's not a complete band.
- I am talking about real life
- What about the other guys? What are you talking about?
- Oh, nevermind
- Well, the way you want to see the people who do that against the part of the world who can see the same time to take the back of the show.
- How old are you?
- Twenty minutes.
- I am not a cop.
- Well, then what?  What are you talking about?
- Go! Take Serge.. and watch your backs.
- You can't stay here and get me there.
- How do I look?
- You know, I don't know.
- Okay, lets do it.
- What are you talking about?
- What?
- I don't know.                                                                                                                                                                                                                                                  
- On my way.
- What do you mean?
- Sure.  Go left.
- You don't know what you're talking about.  You don't have to tell me what you mean.
- Good morning.
- What do yo

In [32]:
ask_my_bot('Oh my God, does this mean you are becoming normal?')
ask_my_bot('You are welcome.')
ask_my_bot('I am talking about real life')
ask_my_bot('Oh, nevermind')
ask_my_bot('Why?')


- Oh my God, does this mean you are becoming normal?
- That's what I wanted to do.  I want to know what happened.
- You are welcome.
- It's not a complete band.
- I am talking about real life
- What about the other guys? What are you talking about?
- Oh, nevermind
- Well, the way you want to see the people who do that against the part of the world who can see the same time to take the back of the show.
- Why?
- I don't know.                                                                                                                                                                                                                                                  
